In [1]:
import Functions as F 
results = "results/results03.csv"
nr_agents = 6
nr = 14
Viz = F.VisualizeAngle(results, nr_agents)
Viz.get_coor()
Viz.plot_angle_in_plane(nr, True)

In [2]:
%matplotlib qt 
rotation_ann = Viz.R_ann
rotation_model = Viz.R_model 
Dij = F.Dijkstras(results, nr_agents, nr, rotation_ann, rotation_model)
Dij.run_dijkstra(6, 26)

In [3]:
Dij.plot_both()

## Plot path on top of image

In [4]:
import SimpleITK as sitk 
import numpy as np 
import os 
import matplotlib.pyplot as plt 

In [24]:
facial_paths = os.listdir("paths/paths_facial")
itkimage = sitk.ReadImage("Cmarl/src/data/images/Data_good_Scaled/" + facial_paths[0][6:])

origin = np.array(list(reversed(itkimage.GetOrigin())))
spacing = np.array(list(reversed(itkimage.GetSpacing())))
itkimage.SetOrigin(origin)
itkimage.SetSpacing(spacing)

mask = sitk.GetArrayFromImage(sitk.ReadImage("paths/paths_facial/" + facial_paths[0]))
im = sitk.GetArrayFromImage(itkimage)
# mask = np.transpose(mask, [0, 2, 1])
im[mask == 1] = 500 

In [27]:
fig, ax = plt.subplots(3, 3, figsize = (14, 14))
ax = ax.flatten()
for i in range(3 * 3): 
    ax[i].imshow(im[0:80, 15:95, i + 27], cmap = "gray")
    ax[i].set_title(f"Slice {i + 27}")
    ax[i].axis("off")
plt.show()

In [18]:
plt.imshow(im[:, :, 71])
plt.show()

In [23]:
path = np.where(mask == 1)
path 

(array([27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 31, 31, 31,
        32, 32, 33, 33, 34, 34, 35, 35, 35, 36, 36], dtype=int64),
 array([37, 34, 35, 36, 38, 38, 33, 35, 36, 37, 39, 32, 34, 47, 31, 34, 46,
        35, 45, 36, 44, 37, 43, 38, 41, 42, 39, 40], dtype=int64),
 array([71, 77, 76, 70, 72, 74, 76, 69, 76, 75, 73, 75, 68, 70, 74, 68, 70,
        68, 70, 68, 71, 68, 70, 67, 69, 70, 68, 68], dtype=int64))